<a href="https://colab.research.google.com/github/lu-lab/frcnn-all-in-one/blob/main/colab/Faster_R_CNN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Faster R-CNN training
---
This notebook will allow you to train a custom object detector using Google's GPU resources. Enable this by going to Runtime -> Change runtime type and select "GPU" from the dropdown menu. This will speed your training time up substantially, but note that Google has a limit on how much of this GPU resource you can use. If you use the GPU resource heavily, you may have to subscribe to a paid plan. In this notebook, we will be fine-tuning the Faster-RCNN network (specifically, the Faster R-CNN Inception ResNet V2) starting from a model pre-trained on the COCO 2017 image set. This pre-trained model is provided in the Tensorflow 2 Model Zoo [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)). 


###Step 00: Copy this folder


---


First off, you won't be able to edit this notebook directly, so **save a copy of this notebook to your own Google Drive!**

**Note**: Following this notebook will require several GB of space in your Google Drive, in addition to whatever space you may need for your annotated image data, or any data you may want to perform inferences on. 


###Step 0: Annotate your data


---


Before you can train your model, you need a **train and test set of annotations** for your images, and the annotations should be converted into the tensorflow-friendly tfrecord format. If you do not have these already, you can use our binder here [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/lu-lab/frcnn-all-in-one/HEAD) to do that! **Once you have finished working through the binder and have downloaded the bounding_boxes.csv and label_map.pbtxt files generated** to your computer, return here. If you used the binder to convert a movie to images for labelling, **download the images folder as well and upload it to the folder this code is in.**

###Step 1: Connect to your Google Drive

---


Now that you're back, make sure your Runtime type is set to GPU (Runtime -> Change runtime type and select "GPU" from the dropdown menu), let's make a few new folders in our current directory by running the following cell. 



In [1]:
# First we need to mount Google drive and gather some dependencies...
from google.colab import drive 
drive.mount('/content/drive')
import os

# NOTE: Modify this path if needed
%cd /content/drive/My\ Drive/Colab\ Notebooks/Faster\ R-CNN/
working_dir = os.getcwd()

# if any of these folders already exist, they will not be made!
!mkdir annotations
!mkdir images
!mkdir exported-model
!mkdir inferencing-results


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Faster R-CNN
mkdir: cannot create directory ‘annotations’: File exists
mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘exported-model’: File exists
mkdir: cannot create directory ‘inferencing-results’: File exists


###Step 2: Install libraries

---

Next we install all the necessary libraries to train our model. At the end of this step, you'll also have several new folders, including Tensorflow, COCO-trained-model, and cocoapi. The Tensorflow directory includes most of the model development pipeline, the cocoapi allows the pipeline to compute metrics describing how well the model performs as we train it, and the COCO-trained-model is our starting model that we will be fine-tuning.

**Note**: If you've already run this notebook once, you do NOT need to re-download the Tensorflow, cocoapi, and COCO-trained-model files, but you DO need to install them and other libraries again if you have stopped the session! If you just need to re-install everything, put a # in front of every line in the code below that has a # at the end of it.

In [ ]:
# You will need to re-install everything if you've restarted your session
%mkdir Tensorflow 
%cd ./Tensorflow
# !git clone https://github.com/tensorflow/models.git #
%cd {working_dir}
!apt-get -qq install -y protobuf-compiler python-pil python-lxml python-tk 
%cd ./Tensorflow/models/research/
!protoc object_detection/protos/*.proto --python_out=.
%cd {working_dir}

# !git clone https://github.com/cocodataset/cocoapi.git #
%cd ./cocoapi/PythonAPI
!make
!cp -r pycocotools /content/drive/My\ Drive/Colab\ Notebooks/Faster\ R-CNN/TensorFlow/models/research/
%cd {working_dir}

%cd Tensorflow/models/research/
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd {working_dir}

%mkdir COCO-trained-model
%cd COCO-trained-model
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
# !tar -xf faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
# !rm faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
# !mv -v faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/* /content/drive/My\ Drive/Colab\ Notebooks/Faster\ R-CNN/COCO-trained-model/ #
# !rmdir faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8 #
%cd {working_dir}

# !cp ./Tensorflow/models/research/object_detection/model_main_tf2.py . #
# !cp ./Tensorflow/models/research/object_detection/exporter_main_v2.py . #

!pip install Cython==0.29.21
!pip install h5py==2.10.0
!pip install opencv-python-headless==4.4.0.44
!pip install scipy
!pip install tensorflow
!pip install tensorboard

%load_ext tensorboard

### Step 3: Generate tfrecord files

---


Now we want to use the bounding_boxes.csv and label_map.pbtxt file to generate tfrecords. Put the 'bounding_boxes.csv' file in the 'annotations' folder and the 'label_map.pbtxt' file in the 'training' folder before running the following cell.








In [11]:
import pandas as pd
import tensorflow as tf

sys.path.append("./Tensorflow/models/research/object_detection")
from object_detection.utils import label_map_util
import generate_tfrecord as gt

def write_tf_record(annotations, tfrecord_path, img_path, label_map):
    tf_writer = tf.io.TFRecordWriter(tfrecord_path)
    for annotation in annotations:
        tf_example = gt.create_tf_example(annotation, img_path, label_map)
        tf_writer.write(tf_example.SerializeToString())

    tf_writer.close()
    output_path = os.path.join(os.getcwd(), tfrecord_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

label_map_path = './training/label_map.pbtxt'
csv_filepath = './annotations/bounding_boxes.csv'

# and now we'll convert annotations to a tfrecord
label_map = label_map_util.get_label_map_dict(label_map_path)
all_annotations = pd.read_csv(csv_filepath)
img_path = './images'

# tfrecord for train annotations
tfrecord_train_path = './annotations/train.record'
train_annotations = all_annotations[all_annotations['test_or_train'].isin(['train'])]
grouped_train_annotations = gt.split(train_annotations, 'filename')
write_tf_record(grouped_train_annotations, tfrecord_train_path, img_path, label_map)

# tfrecord for test annotations
tfrecord_test_path = './annotations/test.record'
test_annotations = all_annotations[all_annotations['test_or_train'].isin(['test'])]
grouped_test_annotations = gt.split(test_annotations, 'filename')
write_tf_record(grouped_test_annotations, tfrecord_test_path, img_path, label_map)

Successfully created the TFRecords: /content/drive/My Drive/Colab Notebooks/Faster R-CNN/./annotations/train.record
Successfully created the TFRecords: /content/drive/My Drive/Colab Notebooks/Faster R-CNN/./annotations/test.record


### Step 4: Train the model

---



Now we will fine-tune the model with our own images and classes. Note that if anything is not in it's proper folder when you run this step, it will fail. You will see a lot of warnings as this step starts to run - not to worry, as long as you don't see an error it should be ok! This step will take a long time, even with the GPU (likely at least a few hours). It is up to you when to stop training (you can stop the kernel altogether, or Ctrl+C), and generally a good rule of thumb is to watch for when the total loss starts to plateau. You should be able to monitor this by watching the Tensorboard that will start when you run the following cell. You can also monitor this in the output above the Tensorboard widget once the first 100 training steps are complete. 

In [4]:
import tensorflow as tf
import datetime

%tensorboard --logdir training/train/
!python model_main_tf2.py --model_dir=training --pipeline_config_path=training/faster_rcnn.config --alsologtostderr

Traceback (most recent call last):
  File "model_main_tf2.py", line 113, in <module>
    tf.compat.v1.app.run()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "model_main_tf2.py", line 110, in main
    record_summaries=FLAGS.record_summaries)
  File "/usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py", line 667, in train_loop
    loss = _dist_train_step(train_input_iter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 855, in _call
  

### Step 5: Export model

---

Once the loss becomes reasonable and you've stopped training, freeze the model and save it to the 'exported-model' folder. You'll see quite a few warnings when you run the following cell, but again no need for concern unless you see an error. Once this is done, double check your 'exported-model' folder. It should now contain a 'checkpoint' and 'saved-model' folder and a 'pipeline.config' file.

In [4]:
!python exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path ./training/faster_rcnn.config \
--trained_checkpoint_dir ./training/ \
--output_directory ./exported-model

2021-04-20 16:40:26.682677: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-20 16:40:31.895720: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-20 16:40:31.936408: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-20 16:40:32.016685: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-20 16:40:32.016772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (acd3afe05462): /proc/driver/nvidia/version does not exist
2021-04-20 16:40:32.017655: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
Instructions for updating:
back_prop=False is deprecated. Consider using tf.

### **Step 6: Test inferencing**

Now our model is frozen, we can use it to inference, or predict the bounding boxes for the classes we trained on with new images. Below are a few different options for how to do this, depending on whether your input data is a list of images, a folder of images, or a movie. 

** NEEDS TO BE UPDATED FOR TF2**

#### **Inferencing for our test images**
First, let's see how the model performs on our annotated test images.

In [10]:
import sys
import glob
import IPython
from IPython.display import Image, display
import cv2
from inference_code import inferencing_tools


path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing-results'
h5_file = os.path.join(save_path, 'test_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# take a look at qualitatively how well the model performs on our test images
test_image_paths = test_images

for path in test_image_paths:
    image_np = cv2.imread(path)
    #inferencing happens in this call
    worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image_np, path)

# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
inferencing_tools.label_all_detections_from_h5(h5_file, test_image_paths, save_path)
for image_name in glob.glob('./inferencing-results/*.JPG'): #assuming JPG
    display(Image(filename=image_name))

NotFoundError: ignored

#### **Inferencing from a folder of images**
This is very similar to the example above. Here we detect all worms and eggs from .jpg images in a folder, saves them to an h5 file, and then overlay the detections on top of the original images.

In [ ]:
path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing'
h5_file = os.path.join(save_path, 'folder_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# take a look at qualitatively how well the model performs on our test images
image_dir = './images'

image_paths = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))
                             and f.endswith('.jpg')]

for path in image_paths:
    image_np = cv2.imread(path)
    #inferencing happens in this call - you can directly use the boxes this returns if you wish. They are also saved in the h5 file
    worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image_np, path)

# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
inferencing_tools.label_all_detections_from_h5(h5_file, image_paths, save_path)
for image_name in glob.glob('./inferencing/*.JPG'): #assuming JPG
    display(Image(filename=image_name))

#### **Inferencing from a movie**
Here's an example that detects both eggs and worms for each frame in a video, saves the detections to an h5 file, and then overlays all detections on top of the original images.

In [ ]:
path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing'
h5_file = os.path.join(save_path, 'movie_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# detect and visualize detections from movie
movie_path = ''
save_file = ''

vid = cv2.VideoCapture(source_data)
idx = 1
while vid.isOpened():
    ret, image = vid.read()
    if ret:
        #inferencing happens in this call
        worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image, idx)
    else:
        break
    print("Processing frame no %s" % i)
    idx += 1
vid.release()

inferencing_tools.label_all_detections_from_h5(h5_file, movie_path, save_file)

# if the input of the inferencing is a video, the output will be a video, otherwise it will be an image
Video(save_file)